In [65]:
import pandas as pd
import geopandas as gpd
import datetime as dt
from shapely.geometry import Point

In [57]:
df_UBC = pd.read_csv("Gbif_UBC.csv", delimiter="\t", low_memory=False)
df_UBC = df_UBC.head(200)

In [58]:
geometry = [Point(xy) for xy in zip (df_UBC['decimalLongitude'], df_UBC['decimalLatitude'])]

In [59]:
geo_df = gpd.GeoDataFrame(df_UBC, geometry = geometry, crs = {'init': 'epsg:4326'})

In [60]:
geo_df = geo_df.rename(columns={'eventDate' : 'time'})

In [62]:
geo_df.to_file("leaflet/UBC_alt.geojson", driver="GeoJSON")

In [128]:
df1 = geo_df.iloc[:,25:28]
#geo_df[['year','month','day']]
#pd.to_datetime(geo_df)
print df1.dtypes
df2 = df1.apply(lambda x : int(x))

day      float64
month    float64
year     float64
dtype: object


TypeError: ("cannot convert the series to <type 'int'>", u'occurred at index day')

In [118]:
df = pd.DataFrame({'year': [2015, 2016],
                       'month': [2, 3],
                       'day': [4, 5]})
print df.dtypes

day      int64
month    int64
year     int64
dtype: object


In [127]:
int(4.729174)

4